In [ ]:
import os
import re
import time
import ollama

# === Setup ===
folder = r"Z:/Nutstore/GPT/SH000852"
pattern = re.compile(r"(?P<date>\d{8})_(?P<type>daily|weekly)\.png", re.IGNORECASE)

# === Step 1: Group files by date ===
files = os.listdir(folder)
date_images = {}

for file in files:
    match = pattern.search(file)
    if not match:
        continue
    date = match.group("date")
    chart_type = match.group("type").lower()
    date_images.setdefault(date, {})[chart_type] = os.path.join(folder, file)

# === Step 2: Run predictions with local image paths via Ollama ===
predictions = {}

for idx, date in enumerate(sorted(date_images)):
    if int(date) < 20140101:
        continue

    daily_path = date_images[date].get("daily")
    weekly_path = date_images[date].get("weekly")
    if not daily_path or not weekly_path:
        continue

    
    prompt = (
        "Please analyze the OHLC candlestick chart, volume, and MACD indicators to predict tomorrow's price trend.\n"
        "In the candlestick chart, green candles indicate a bullish day (close > open), while red/orange candles indicate a bearish day (close < open).\n"
        "The first image is the daily chart, and the second is the weekly chart.\n\n"
        "Instructions:\n"
        "1. As an experienced technical analyst, use your expertise to make a judgment.\n"
        "2. Provide the conclusion directly, in one of the following formats (no explanation needed):\n"
        "- The price may rise tomorrow. Probability: 0.x\n"
        "- The price may fall tomorrow. Probability: -0.x\n"
        "- Tomorrow is uncertain or sideways. Probability: 0\n\n"
        "**Note: Only one of the three options above should be provided.**"
    )

    try:
        response = ollama.chat(
            # model="qwen2.5vl:32b",  # Or whichever model you have running
            model="llama3.2-vision:11b",  # Or whichever model you have running
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                    "images": [daily_path, weekly_path]  # Send images directly from local files
                }
            ]
        )
        predictions[date] = response['message']['content']
        print(f"✅ {date} processed")

    except Exception as e:
        print(f"❌ Error on {date}: {e}")

    # Optional pause
    if (idx + 1) % 50 == 0:
        print(f"⏸️ Pausing after {idx + 1} requests...")
        time.sleep(10)


In [ ]:
for date in sorted(predictions):
    print("=" * 40)
    print(f"📅 Date: {date}")
    print(f"🧠 Prediction and Reasoning:\n{predictions[date]}\n")

In [ ]:
import re
import json
import pandas as pd

# === Step 1: Parse Function ===
def parse_prediction(raw_text):
    """
    Extract direction and numeric prediction from model output like:
    - The price may rise tomorrow. Probability: 0.6
    - The price may fall tomorrow. Probability: -0.4
    - Tomorrow is uncertain or sideways. Probability: 0
    """
    # Extract numeric prediction
    prob_match = re.search(r"Probability[:：]?\s*(-?\d+\.?\d*)", raw_text)
    if prob_match:
        try:
            pred_y = float(prob_match.group(1))
        except ValueError:
            pred_y = 0.0
    else:
        pred_y = 0.0

    # Determine direction
    if "rise" in raw_text.lower():
        direction = "Up"
    elif "fall" in raw_text.lower():
        direction = "Down"
    elif "uncertain" in raw_text.lower() or "sideways" in raw_text.lower():
        direction = "Sideways"
    else:
        direction = "Unknown"

    return {
        "direction": direction,
        "pred_y": pred_y
    }

# === Step 2: Process Predictions ===
structured_preds = {}

for date, raw in predictions.items():
    parsed = parse_prediction(raw)
    structured_preds[date] = {
        "raw": raw,
        "direction": parsed["direction"],
        "pred_y": parsed["pred_y"]
    }

# === Step 3: Save to JSON ===
json_path = "z:/SVXYforecast_predictions_with_probqwen32b.json"

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(structured_preds, f, ensure_ascii=False, indent=2)

print(f"✅ Saved to {json_path}")

# === Step 4: Read Back and Save to CSV ===
with open(json_path, "r", encoding="utf-8") as f:
    preds = json.load(f)

df = pd.DataFrame([
    {
        "date": date,
        "direction": info["direction"],
        "pred_y": info["pred_y"],
        "raw_text": info["raw"]
    }
    for date, info in preds.items()
])

csv_path = "z:/forecast_predictions_with_probqwen32bENGall.csv"
csv_path = "z:/forecast_predictions_with_probllama11bENGall.csv"
df.to_csv(csv_path, index=False)
print(f"📄 CSV exported to {csv_path}")